In [1]:
import os
import time

import torch
import transformers
from dotenv import load_dotenv
from langchain.cache import SQLiteCache
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    CharacterTextSplitter,
    MarkdownHeaderTextSplitter,
    MarkdownTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores import Qdrant
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
)


## Build LLM Model

In [2]:
model_name = "MediaTek-Research/Breeze-7B-32k-Instruct-v1_0"

tokenizer = AutoTokenizer.from_pretrained(
    model_name, cache_dir="../llm_model/", trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
streamer = TextStreamer(tokenizer=tokenizer, skip_prompt=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="../llm_model/",
    device_map="auto",
    low_cpu_mem_usage=True,  # try to limit RAM
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),  # load model in low precision to save memory
    # attn_implementation="flash_attention_2",
)
# Building a LLM QNA chain
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.5,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=2048,
    streamer=streamer,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
huggingFacePipeline = HuggingFacePipeline(pipeline=text_generation_pipeline)

## Read Document

### MarkdownHeaderTextSplitter

In [4]:
with open("../docs/childcare_subsidy_qa.md", "r", encoding="utf-8") as f:
    data = f.read()

data

'# 未滿 2 歲兒童托育補助問答集 \n\n(112 年 1 月 1 日起適用)\n\n## 新制規定\n\n### 未滿 2 歲兒童托育補助發放對象？(作業要點第 12 點)\n\n一、實際年齡未滿2歲的我國籍兒童，且請領當時符合下列情形者： \n(一)委託人之幼兒完成出生登記或戶籍登記。 \n(二)送托與政府簽約之公設民營托嬰中心、社區公共托育家園、私立托嬰中心及居家托育人員(保母)。 \n(三)每週送托時數達30小時以上。\n(四)未領取育兒津貼。 \n(五)未經政府公費安置。 \n(六)不得指定一對一收托。但發展遲緩、身心障礙、罕見疾病或有其他特殊狀況之幼兒，不在此限。\n\n二、110年8月起取消「育嬰留職停薪津貼、弱勢兒少生活津貼不得同時領取托育補助」規定，另112年起取消綜合所得稅率未達20%之規定。換言之，如有送托公設民營托嬰中心、社區公共托育家園、準公共托嬰中心或準公共保母，且未申領育兒津貼或接受政府安置收容之未滿2歲兒童，都可申請托育補助。\n\n### 未滿 2 歲兒童托育補助發放金額？\n\n一、補助金額： \n(一) 公共化托育補助：111年8月起每月5,500元起。 \n(二) 準公共托育補助：111年8月起每月8,500元起。 \n\n二、第2名子女即加發：第2名子女加發1,000元、第3名以上子女則加發2,000元。(備註：胎次計算，請參考Q14)\n\n三、擴大補助對象：取消綜合所得稅率未達20%、育嬰留職停薪津貼、弱勢兒童生活類補助不得同時領取的規定。如果您正在領取軍保、公保、就保育嬰留職停薪津貼或其他弱勢兒童少年生活類補助，均可以申請托育補助。\n\n112年1月起每名兒童每月發放金額 \n單位：元/月 \n             出生 \n             排序 \n家庭 \n類別 \n公共化 \n(公設民營、社區公共托育家\n園) \n準公共 \n (簽約之私立托嬰中心及居家托育) \n第1名 \n子女 \n第2名 \n子女 \n第3名 \n以上子女 \n第1名 \n子女 \n第2名 \n子女 \n第3名 \n以上子女 \n一般家庭 5,500 6,500 7,500 8,500 9,500  10,500 \n中低收入戶 7,500 8,500 9,500 10,500 11,500 12,500 \n低收入戶

In [5]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

md_splits = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
).split_text(data)

print(f"md_splits len: {len(md_splits)}\n")
for split in md_splits:
    print(split)


md_splits len: 19

page_content='(112 年 1 月 1 日起適用)' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集'}
page_content='一、實際年齡未滿2歲的我國籍兒童，且請領當時符合下列情形者：\n(一)委託人之幼兒完成出生登記或戶籍登記。\n(二)送托與政府簽約之公設民營托嬰中心、社區公共托育家園、私立托嬰中心及居家托育人員(保母)。\n(三)每週送托時數達30小時以上。\n(四)未領取育兒津貼。\n(五)未經政府公費安置。\n(六)不得指定一對一收托。但發展遲緩、身心障礙、罕見疾病或有其他特殊狀況之幼兒，不在此限。  \n二、110年8月起取消「育嬰留職停薪津貼、弱勢兒少生活津貼不得同時領取托育補助」規定，另112年起取消綜合所得稅率未達20%之規定。換言之，如有送托公設民營托嬰中心、社區公共托育家園、準公共托嬰中心或準公共保母，且未申領育兒津貼或接受政府安置收容之未滿2歲兒童，都可申請托育補助。' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集', 'Header 2': '新制規定', 'Header 3': '未滿 2 歲兒童托育補助發放對象？(作業要點第 12 點)'}
page_content='一、補助金額：\n(一) 公共化托育補助：111年8月起每月5,500元起。\n(二) 準公共托育補助：111年8月起每月8,500元起。  \n二、第2名子女即加發：第2名子女加發1,000元、第3名以上子女則加發2,000元。(備註：胎次計算，請參考Q14)  \n三、擴大補助對象：取消綜合所得稅率未達20%、育嬰留職停薪津貼、弱勢兒童生活類補助不得同時領取的規定。如果您正在領取軍保、公保、就保育嬰留職停薪津貼或其他弱勢兒童少年生活類補助，均可以申請托育補助。  \n112年1月起每名兒童每月發放金額\n單位：元/月\n出生\n排序\n家庭\n類別\n公共化\n(公設民營、社區公共托育家\n園)\n準公共\n(簽約之私立托嬰中心及居家托育)\n第1名\n子女\n第2名\n子女\n第3名\n以上子女\n第1名\n子女\n第2名\n子女\n第3名\n以上子女\n一般家庭 5,500 6,500 7,500 8,500 9,500  10,500\n中

### RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=10,
    separators=[
        "\n\n",
        "\n",
        " ",
        "。",
        "，",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)
chunked_documents = text_splitter.split_documents(md_splits)
print(f"\nchunked_documents len: {len(chunked_documents)}")
for chunk in chunked_documents:
    print(chunk)



chunked_documents len: 123
page_content='(112 年 1 月 1 日起適用)' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集'}
page_content='一、實際年齡未滿2歲的我國籍兒童，且請領當時符合下列情形者：' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集', 'Header 2': '新制規定', 'Header 3': '未滿 2 歲兒童托育補助發放對象？(作業要點第 12 點)'}
page_content='(一)委託人之幼兒完成出生登記或戶籍登記。' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集', 'Header 2': '新制規定', 'Header 3': '未滿 2 歲兒童托育補助發放對象？(作業要點第 12 點)'}
page_content='(二)送托與政府簽約之公設民營托嬰中心、社區公共托育家園' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集', 'Header 2': '新制規定', 'Header 3': '未滿 2 歲兒童托育補助發放對象？(作業要點第 12 點)'}
page_content='、社區公共托育家園、私立托嬰中心及居家托育人員(保母)' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集', 'Header 2': '新制規定', 'Header 3': '未滿 2 歲兒童托育補助發放對象？(作業要點第 12 點)'}
page_content='。' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集', 'Header 2': '新制規定', 'Header 3': '未滿 2 歲兒童托育補助發放對象？(作業要點第 12 點)'}
page_content='(三)每週送托時數達30小時以上。\n(四)未領取育兒津貼。' metadata={'Header 1': '未滿 2 歲兒童托育補助問答集', 'Header 2': '新制規定', 'Header 3': '未滿 2 歲兒童托育補助發放對象？(作業要點第 12 點)'}
page_content='(五)未經政府公費安置。' 

## Build Retriever

In [7]:
start_time = time.time()

sentence_transformer_model_root = "../sentence_transformer_model"
sentence_transformer_model = "multi-qa-mpnet-base-dot-v1"

# Load chunked documents into the Qdrant index
db = Qdrant.from_documents(
    chunked_documents,
    HuggingFaceEmbeddings(model_name=os.path.join(
            sentence_transformer_model_root, sentence_transformer_model
        )),
    location=":memory:", # Local mode with in-memory storage only
)

retriever = db.as_retriever(search_type="mmr")
retrieval_chain = RetrievalQA.from_llm(llm=huggingFacePipeline, retriever=retriever)

print("\ntime: %.2fs\n" % (time.time() - start_time))



time: 7.94s



## Ask LLM

In [8]:
query = f"申請未滿 2 歲托育補助需準備哪些文件？"

start_time = time.time()
response = retrieval_chain.invoke(query)

# Extract only the necessary part from the response
# start_idx = response.find("Helpful Answer: ")
# if start_idx != -1:
#     response = response[start_idx + len("Helpful Answer: ") :]

print(f"response: {response}")
print("\nInference time: %.2fs\n" % (time.time() - start_time))

e:\dev\auray\.venv\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:674: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


為申請未滿 2 歲托育補助，需要準備以下文件：1. 名下房屋及土地所有權狀或租賃契約、1. 家庭生活支出明細、1. 全戶人口最新戶口資料、1. 家長工作證明或收入證明、1. 低收入戶證明 (如有)。</s>
response: {'query': '申請未滿 2 歲托育補助需準備哪些文件？', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext:\n| 情形 | 應備文件 |\n\nContext:\n如果正在領取未滿 2 歲托育補助，兒童滿 2 歲未滿 3\n\nContext:\n2 名以上(含)子女證明文件，方便核定機關查調比對。\n\nContext:\n低收入戶\n\nQuestion: 申請未滿 2 歲托育補助需準備哪些文件？\nHelpful Answer: 為申請未滿 2 歲托育補助，需要準備以下文件：1. 名下房屋及土地所有權狀或租賃契約、1. 家庭生活支出明細、1. 全戶人口最新戶口資料、1. 家長工作證明或收入證明、1. 低收入戶證明 (如有)。"}

Inference time: 27.65s

